### Installs 
##### - in dev container for codespaces

In [ ]:
# %pip install hvplot
# %pip install plotly
# %pip install holoviews
# %pip install xarray
# %pip install panel
# %pip install matplotlib
# %pip install jupyter_bokeh
# %pip install cognite-sdk
# %pip install cognite

### Imports

In [ ]:
import numpy as np

import datetime as dt
import panel as pn

pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df

import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas


pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')

### cognite credentials

In [ ]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"

# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"

In [ ]:
from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter

In [ ]:
display(f'{client.version=} / {client.config.project=}')
# print(client.iam.token.inspect())

In [ ]:
#test

In [ ]:
wellcom_dataset = "src:015:wellcom"
edm_dataset = "src:012:edm"
witsml_dataset = "src:014:witsmldata"

In [ ]:
active_wellbore = "active:wellbore"
match_witsml = "match:witsml:wellcom" # sensor data only for johan svendrup
match_edm = "match:edm:wellcom"

# defien and show datasets

In [ ]:
#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
active_wellbores

In [ ]:
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)
witsml_db

## make to padnas

In [ ]:
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)
pd_active_wellbores

In [ ]:
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml

### query pandas witsml

In [ ]:
_pd_witsml.columns

## flatten witsml data

In [ ]:
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'])], axis=1)



In [ ]:
_pd_witsml_flattened.columns

In [ ]:
_pd_witsml_flattened = _pd_witsml_flattened[['id', 'uidWell', 'uidWellbore',  'nameWellbore','external_id', 'name', 'is_string', 'unit', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType', 'name', 'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid', 'unit']]
_pd_witsml_flattened

In [ ]:
_pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']['external_id'].iloc[0]

In [ ]:
_pd_witsml_flattened['nameWell'].nunique()

In [ ]:
_df_name_is_cempumppr = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'CEMPUMPPR']

In [ ]:
_df_name_is_cempumppr.sort_values(by=['startDateTimeIndex']).head(80)

In [ ]:
len(_df_name_is_cempumppr)

In [ ]:
len(_df_name_is_cempumppr['nameWellbore'].unique())

## find sensorid to base real time data on

In [ ]:
_df_name_is_bitdep = _pd_witsml_flattened[_pd_witsml_flattened['description'] == 'BITDEP']

In [ ]:
_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-18']

## set sensorid

In [ ]:
_sel_sensorid = int(_df_name_is_bitdep[_df_name_is_bitdep['nameWellbore'] == 'NO 16/2-D-15'].id.iloc[0])
_sel_sensorid 

#### make list of all sensorids to use in panel

In [ ]:
_sensorids_list = pd.Series(_df_name_is_bitdep['nameWellbore']).unique().tolist()
# _sensorids_list

In [ ]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list


#### panel choose _sel_sensorid

In [ ]:
# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')

# def update_indicator(event, clicks):
#     if not event:
#         return
    
#     indicator.value = not indicator.value
#     text.value = f"you have clicked me {clicks} times"

# pn.bind(update_indicator, test_widget, test_widget.param.clicks, watch=True)

# pn.Column(pn.Row(test_widget, text),indicator)

In [ ]:
# _sensorids_list = [{1:"fefe"}, {2 :"foo"}]

In [ ]:
sr2 = pd.Series(pd.Series(_df_name_is_bitdep['nameWellbore']).tolist(), index = _df_name_is_bitdep['id'])
d = {key: value for key, value in sr2.items()}
_sensorids_list = d
_sensorids_list



_sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list)



# def _update__sel_sensorid
# @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
def _update__sel_sensorid( self, value):
    _sel_sensorid = value
    return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

In [ ]:
_sel_sensorid =_sel_sensorid_widget.param.values().__getitem__('value')
_sel_sensorid
## TODO why is _sel_sensorid not updating automatically?

In [ ]:
_sel_sensorid

In [ ]:
_sel_sensorid = 746172675242144

## TIME SERIES QUERY based on _sel_sensorid

In [ ]:
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

In [ ]:
client.time_series.data.retrieve(id=746172675242144,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1)

In [ ]:
_df_real_time = client.time_series.data.retrieve(
    id=_sel_sensorid,
    start=MIN_TIMESTAMP_MS,
    end=MAX_TIMESTAMP_MS + 1) 
_df_real_time


## to pandas

In [ ]:
_pd_real_time = _df_real_time.to_pandas( )
# make Value columsn or easier filtering
_pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]

In [ ]:
_pd_real_time.sort_index(ascending=True)

In [ ]:
_pd_real_time

In [ ]:

_pd_real_time_plot = _pd_real_time
# da['new_index'] = str(da.index)
# da = da.drop(columns=['dw:witsml:log.1ac57fe7-4c90-4bc7-a350-8d17f077bb32.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.05424d07-6a7d-4ef2-acc1-2c2a6c2eb062.bitdep||v1'])
# da.set_index('new_index')

In [ ]:
_pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
_pd_real_time_plot['Date'] = _pd_real_time_plot.index
_pd_real_time_plot

In [ ]:
# from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature as sst

# sst.hvplot()

In [ ]:
min(_pd_real_time_plot['Date'])
# Timestamp('2022-01-26 07:42:42')

In [ ]:
max(_pd_real_time_plot['Date'])

In [ ]:
# _pd_real_time_plot[pd.isnull(_pd_real_time_plot).any(axis=1)]

## plot real time data

In [ ]:
# _pd_real_time_plot.hvplot(height=1000,
#     width=500, title='Real Time',
#     x='Value',
#     y=['Date'], 
#     ylim=(min(_pd_real_time_plot['Date']), max(_pd_real_time_plot['Date']))
# ).opts(invert_yaxis=True)

## plot with panel

In [ ]:
# sr2 = pd.Series(pd.Series(_pd_witsml_flattened['id']).tolist(), index = _pd_witsml_flattened['id'])
# # d = {key: value for key, value in sr2.items()}
# # _sensorids_list = d
# _sensorids_list = pd.Series(_pd_witsml_flattened['id']).tolist()



# # create the Select widget
# _sel_sensorid_widget = pn.widgets.Select(name='Select', options=_sensorids_list, width=200)



# # def _update__sel_sensorid
# # @param.depends(_sel_sensorid_widget.param.values().__getattribute__('value'))
# def _update__sel_sensorid(value):
#     _sel_sensorid = value
#     # do something with _sel_sensorid
#     return _sel_sensorid

# bind _sel_sensorid to event _sel_sensorid_widget and display together with _sel_sensorid_widget
# pn.Row(_sel_sensorid_widget, _sel_sensorid_widget.param.values().__getitem__('value'), pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().__getitem__('value'), watch=True )).servable() 

# pn.Row(_sel_sensorid_widget, pn.bind(_update__sel_sensorid, _sel_sensorid_widget,value=_sel_sensorid_widget.param.values().items())).servable()

#### make plot

In [ ]:

# # make plot of real time data
# plot =_pd_real_time_plot.hvplot(height=1000,
#     width=500, title='Real Time',
#     x='Value',
#     y='Date', 
#     ylim=(min(_pd_real_time_plot['Date']), max(_pd_real_time_plot['Date']))
# ).opts(invert_yaxis=True)




#### show panel

In [ ]:

# # bind the update function to the widget
# _sel_sensorid_widget.param.watch(_update__sel_sensorid, 'value')

# # display the widget and plot side by side
# pn.Row(_sel_sensorid_widget, plot,_sel_sensorid_widget.value).servable()

## trying with plotly

In [ ]:
# pn.pane.Perspective(
#     _pd_real_time,columns=['Value']
# ).servable()

## trying to bind properly

In [ ]:
# pn.extension(design='material')
 
# csv_file = ("https://raw.githubusercontent.com/holoviz/panel/main/examples/assets/occupancy.csv")

# data = pd.read_csv(csv_file, parse_dates=["date"], index_col="date")
 
# data.tail()
 
 
 
# def transform_data(variable, window, sigma):
#     ''' Calculates the rolling average and the outliers '''
#     avg = data[variable].rolling(window=window).mean()
#     residual = data[variable] - avg
# #     std = residual.rolling(window=window).std()
# #     outliers = np.abs(residual) > std * sigma
# #     return avg, avg[outliers]
 
# def create_plot(df):
#     ''' Plots data '''
#     # avg, highlight = transform_data(variable, window, sigma)
#     plot = df.hvplot(height=1000,
#         width=500, title='Real Time',
#         x='Value',
#         y=['Date'], 
#         ylim=(min(df['Date']), max(df['Date']))
#         ).opts(invert_yaxis=True)
#     return plot
#     # return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
#     #     color="orange", padding=0.1, legend=False
#     # )
 
 
 
 
# variable_widget = pn.widgets.Select(name="variable", options=list(_pd_real_time_plot.columns))
# window_widget = pn.widgets.IntSlider(name="window", value=30, start=1, end=60)
# sigma_widget = pn.widgets.IntSlider(name="sigma", value=10, start=0, end=20)
# test_widget = pn.widgets.Button(name="testing", button_type="primary")
 
 
# # variable_widget.servable()
 
 
 
# # create_plot(data=_pd_real_time_plot, window=20, sigma=10)
 
 
 
# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')
 
# def update_indicator(event, clicks):
#     if not event:
#         return
   
#     indicator.value = not indicator.value
#     text.value = f"you have clicked me {clicks} times"
 
# # pn.bind(update_indicator, test_widget, test_widget.param.clicks, watch=True)
 
# pn.Column(pn.Row(test_widget, text),indicator)
 
 
 
 
 
 
# bound_plot = pn.bind(create_plot, variable=variable_widget, window=window_widget, sigma=sigma_widget)
 
 
 
 
 
# first_app = pn.Column(variable_widget, window_widget, test_widget, sigma_widget, bound_plot)
# first_app.servable()

In [ ]:
# #plot data 
# def create_plot(df, wellbore, sensortype, df_witsml):
#     ''' Plots data '''
#     # avg, highlight = transform_data(variable, window, sigma)
#     # id, df = _make_pd_real_time_plot(_get_sensorid(_pd_witsml_flattened))
#     # df = _new_plot
#     _w_wellbores_rx = _w_wellbores_select(df_witsml).rx()
    
#     plot = df.hvplot(height=1000,
#         width=500, title= 'Real Time data for ' + wellbore + " " + sensortype,
#         x='Value',
#         y=['Date'], 
#         ylim=(min(df['Date']), max(df['Date']))#,
        
#         #widgets={'nameWellbore' : _w_wellbores}
#         # ).opts(invert_yaxis=True)
#     )
    
#     _pn_plot = pn.Row(pn.Column(_w_wellbores_rx ),pn.rx(plot))
#     return plot
#     # return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
#     #     color="orange", padding=0.1, legend=False
#     # )





# # make datasets
# def _make_pd_real_time_plot(sensorid) :
#     """
#     make pd of real time dat to be plotted for given sensorid
#     Args: sensorid - the id of the sensor
#     Returns: the id column and the pandas to be plotted
#     """
#     _pd_real_time_plot = _make_pd_real_time(sensorid)
#     _realtimedata_id = _pd_real_time_plot.columns
#     _pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
#     _pd_real_time_plot['Date'] = _pd_real_time_plot.index
#     return _realtimedata_id, _pd_real_time_plot

# def _make_pd_real_time(sensorid):
#     """make pd of real time data for given sensorid"""
#     _pd_real_time = _get_df_real_time(sensorid).to_pandas()
#     # make Value columsn or easier filtering
#     _pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]
#     return _pd_real_time

# def _get_df_real_time(sensorid):
#     """make cognite client of real time data for given sensorid"""
#     _df_real_time = client.time_series.data.retrieve(
#         id=sensorid, # TODO make function to choose
#         start=MIN_TIMESTAMP_MS,
#         end=MAX_TIMESTAMP_MS + 1) 
#     return _df_real_time   

# # get sensor id
# def _get_sensorid(df):
#     """
#         Args: df on the format of _pd_witsml_flattened
#         Returns: the sensor id form the _pd_witsml_flattened based on chosen wellbore and chosen sensortype
#     """
#     # _chosen_wellbore = str(_w_wellbores_select(df).value)
#     _chosen_wellbore = _w_wellbores_select(df).value
#     _chosen_sensortype= str(_w_sensortypes_select(df).value)
#     # _chosen_sensortype= _chosen_sensortype.value
#     _sensor_row = df[(df['nameWellbore'] == _chosen_wellbore) & (df['uid'] == _chosen_sensortype)]
#     _sensorid = int(_sensor_row['id'].iloc[0])
#     _sensortype = _sensor_row['uid'].iloc[0]
#     _wellbore = _sensor_row['nameWellbore'].iloc[0]
#     return _sensorid, _wellbore, _sensortype




# # make list for the widgets

# # def _make_sensorids_list(df):
# #     sr2 = pd.Series(pd.Series(df['id']).tolist(), index = df['id'])
# #     d = {key: value for key, value in sr2.items()}
# #     _sensorids_list = d
# #     return _sensorids_list


# def _make_wellbores_list(df):
#     # sr2 = pd.Series(pd.Series(df['nameWellbore']).unique().tolist(), index = df['id'])
#     _wellbores_list = pd.Series(df['nameWellbore']).unique().tolist()
#     # d = {key: value for key, value in sr2.items()}
#     # _sensorids_list = d
#     return _wellbores_list

# def _make_sensortypes_list(df):
#     # sr2 = pd.Series(pd.Series(df['uid']).unique().tolist(), index = df['id']) # TODO check that uid is same as name aka sensortype
#     _sensortypes_list = pd.Series(df['uid']).unique().tolist()
#     # d = {key: value for key, value in sr2.items()}
#     # _sensorids_list = d
#     return _sensortypes_list




# ## make widgets
# def _w_wellbores_select(df):
#     """
#     Args: df on the format of _pd_witsml_flattened
#     """
#     _sel_sensorid_widget = pn.widgets.Select(name='Select', options=_make_wellbores_list(df))
#     return _sel_sensorid_widget

# def _w_sensortypes_select(df):
#     """
#     Args: df on the format of _pd_witsml_flattened
#     """
#     _sel_sensortype_widget = pn.widgets.Select(name='Select', options=_make_sensortypes_list(df))
#     return _sel_sensortype_widget

# # get widget values
# def _get_wellbore():
#     return _w_wellbores_select().value

# def _get_sensortype():
#     return _w_sensortypes_select().value

# # @pn.depends(_w_wellbores_select(_pd_witsml_flattened))
# # def _new_plot():
#     # return _w_wellbores_select(_pd_witsml_flattened).value

# def b(event):
#     print("hellp")



# # define the update function
# def update_plot(event):
#     sensorid = _get_sensorid(_pd_witsml_flattened)
#     id, data = _make_pd_real_time_plot(sensorid)
#     new_plot = create_plot(data, _pd_witsml_flattened)
#     plot.object = new_plot.object

# # _w_wellbores_select(df)



# # _w_wellbores_select =  select(_pd_witsml_flattened)
# # _w_sensortypes_select = pn.widgets.Select(name='Select Sensor Type')

# # define the update function
# def update_plot(wellbore):
#     sensorid = _get_sensorid(_pd_witsml_flattened, wellbore)
#     id, data = _make_pd_real_time_plot(sensorid)
#     new_plot = create_plot(data, wellbore, _w_sensortypes_select.value, _pd_witsml_flattened)
#     plot.object = new_plot.object

# # bind the update function to the widget values
# _w_wellbores_select.param.value = pn.bind(update_plot, wellbore=_w_wellbores_select.value)

# _w_sensortypes_select.param.value = pn.bind(update_plot, wellbore=_w_wellbores_select.value)

# # display the widgets and plot
# pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()



# # create the Select widgets
# # _w_wellbores_select_widget = _w_wellbores_select(_pd_witsml_flattened)
# # _w_sensortypes_select_widget = pn.widgets.Select(name='Select Sensor Type', options=_make_sensortypes_list(_pd_witsml_flattened))

# # bind the update function to the widget values
# #_w_wellbores_select(_pd_witsml_flattened).param.watch(update_plot, 'value')
# # pn.bind(update_plot, pn.widgets.Select(name='Select Wellbore', options=_make_wellbores_list(_pd_witsml_flattened)))
# # _w_sensortypes_select.param.watch(update_plot, 'value')

# # \pn.bind(update_plot, event='new')
# # pn.bind(update_plot, event='new')

# # pn.bind(update_plot, _w_wellbores_select_widget,  watch=True)

# # _w_wellbores_select.param.value.bind(update_plot)
# # _w_sensortypes_select.param.value.bind(update_plot)

In [ ]:
#### REWQRITE PROPER NEW ONES


# make lists to use in widgets
def _make_wellbores_list(df):
    _wellbores_list = pd.Series(df['nameWellbore']).unique().tolist()
    return _wellbores_list

def _make_sensortypes_list(df):
    _sensortypes_list = pd.Series(df['uid']).unique().tolist()
    return _sensortypes_list




## make widgets
def _w_wellbores_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _wellbore = pn.widgets.Select(name='Select wellbore', options=_make_wellbores_list(df))
    return _wellbore

def _w_sensortypes_select(df):
    """
    Args: df on the format of _pd_witsml_flattened
    """
    _sensor_type = pn.widgets.Select(name='Select sensor type', options=_make_sensortypes_list(df))
    return _sensor_type


# Set variables
_w_sensortype = _w_sensortypes_select(_pd_witsml_flattened)
_w_wellbore = _w_wellbores_select(_pd_witsml_flattened)







#plot data 
def create_plot(_interactive, test="false"):#, wellbore, sensortype, df_witsml):
    ''' Plots data '''    
    # _sensortype = _w_sensortype.param.get_param_values().__getitem__(24)[1]
    print("below is from create_plot")
    _sensortype = _w_sensortype
    _wellbore = _w_wellbore
    # print(_sensortype)
    line_plot = _interactive.hvplot(height=1000,

        width=500, title= 'Real Time data for ' + " ",# + str(dict(_w_sensortype.param.values().items())['value']),# + _wellbore,
        x='Value',
        xlabel = _sensortype,
        # hover_cols=['Description'],
        # y=['Date'],
        y=['Date_Seconds'],

        autorange="y"
        # value_label = 'Date'
        # ,ylim=(min(df['Date']), max(df['Date']))#,
    )
    constant_x = 10
    # x=pd.Series([constant_x] * num_rows)
    # scatter_plot = _interactive.hvplot(y=['DescriptionDate'],hover_cols=['Description'], kind='scatter')
    # combined_plot  =plot * scatter_plot

        ### THIS ONE - edit with pd.Timestamp() instead
    # Define the coordinates of the rectangles
    # rectangles = [
    #     {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    #     {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    #     {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
    # ]


    _df_rectangles = pd.DataFrame({
        'x': [2000000000000.00, 2000000000000.00, 2000000000000.00],
        'ystart': [pd.Timestamp('2022-11-24 05:27:00'), pd.Timestamp('2022-11-29 05:25:00'), pd.Timestamp('2022-12-25 05:29:00')],
        'yend': [pd.Timestamp('2022-11-25 05:27:00'), pd.Timestamp('2022-12-02 05:27:00'), pd.Timestamp('2022-12-29 05:27:00')],
        'width': [20000000000000.00, 20000000000000.00, 20000000000000.00],
        'label': ['Rectangle 1 - U', 'Rectangle 2 - D', 'Rectangle 3 - U']
    })

    _df_rectangles['ystart_seconds'] =  _df_rectangles['ystart'].apply(lambda x: x.timestamp())
    _df_rectangles['yend_seconds'] =  _df_rectangles['yend'].apply(lambda x: x.timestamp())

    # # Convert the y values to Unix timestamps
    # for rectangle in _df_rectangles:
    #     rectangle['ystart'] = rectangle['ystart'].timestamp()
    #     rectangle['yend'] = rectangle['yend'].timestamp()


    # setting global values for styling graps
    HEIGHT = 500
    WIDTH = 500
    
    # Create a list to store the polygons and text labels
    plots = []

    # Add the rectangles to the plot
    for rectangle in _df_rectangles:
        # Define the vertices of the rectangle
        vertices = [(rectangle['x'], rectangle['ystart']), (rectangle['x'] + rectangle['width'], rectangle['ystart']),
                    (rectangle['x'] + rectangle['width'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend'])),
                    (rectangle['x'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']))]
        

    # Convert the y-coordinates to float values
    vertices = [(x,ystart) for x, ystart in vertices]

    # set colors of rectanlges
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5, color = fill_color)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']) / 2, rectangle['label']).opts(color='black', xaxis=None, height =HEIGHT, width = WIDTH) # yaxis=None,
    
    # Add the label to the list of plots
    plots.append(label)
    # Combine all the plots into a single plot
    combined_plot = line_plot + hv.Overlay(plots).opts(yaxis )
    # Display the plot
    # combined_plot = hv.Layout([line_plot, hv.Overlay(plots)]).cols(2).opts(shared_axes=True)

    return combined_plot#(combined_plot).opts(shared_axes=True)

# Define a function to update the labels
def update_labels( plot, well, sensor, y_label= 'Date'):
    plot.opts(xlabel = sensor, title=f"Real time data for {well} - {sensor}")








# make datasets
def _make_pd_real_time_plot(sensorid) :
    """
    make pd of real time dat to be plotted for given sensorid
    Args: sensorid - the id of the sensor
    Returns: the id column and the pandas to be plotted
    """
    _pd_real_time_plot = _make_pd_real_time(sensorid)
    _realtimedata_id = _pd_real_time_plot.columns
    _pd_real_time_plot = _pd_real_time_plot.drop(_pd_real_time_plot.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
    _pd_real_time_plot['Date'] = pd.to_datetime(_pd_real_time_plot.index)
    _pd_real_time_plot['Date_Seconds'] = _pd_real_time_plot['Date'].apply(lambda x: x.timestamp())
    # _pd_real_time_plot['Date'] =_pd_real_time_plot.index
    return _realtimedata_id, _pd_real_time_plot

def _make_pd_real_time(sensorid):
    """make pd of real time data for given sensorid"""
    _pd_real_time = _get_df_real_time(sensorid).to_pandas()
    # make Value columsn or easier filtering
    _pd_real_time['Value'] = _pd_real_time[_pd_real_time.columns]
    return _pd_real_time

def _get_df_real_time(sensorid):
    """make cognite client of real time data for given sensorid"""
    _df_real_time = client.time_series.data.retrieve(
        external_id= None,
        id=sensorid, # TODO make function to choose
        start=MIN_TIMESTAMP_MS, # TODO change to acutal vlaues,min(df['Date']) 
        end=MAX_TIMESTAMP_MS + 1) 
    return _df_real_time   

# get sensor id
def _get_sensorid(df, wellbore, sensortype):
    """
        Args: df on the format of _pd_witsml_flattened
        Returns: the sensor id form the _pd_witsml_flattened based on chosen wellbore and chosen sensortype
    """
    _sensor_row = df[(df['nameWellbore'] == wellbore) & (df['uid'] == sensortype)]
    _sensorid = _sensor_row['id'].iloc[0]
    return _sensorid





In [ ]:
_w_wellbores_select(_pd_witsml_flattened).value

In [ ]:
# display the widgets and plot
# pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()

In [ ]:

# sensorid, wellbore, sensor =  _get_sensorid(_pd_witsml_flattened)
# id, data = _make_pd_real_time_plot(sensorid)


# # #btn = pn.widgets.Button(name = "click me", button_type = 'primary')

# plot = create_plot(data, wellbore, sensor, _pd_witsml_flattened)
# # plot.param.trigger('draw')
# plot

In [ ]:
# import panel as pn
# import pandas as pd
# import xarray as xr
# import hvplot.xarray

# # create the Select widgets
# _w_wellbores_select = pn.widgets.Select(name='Select Wellbore',  options=_make_wellbores_list(_pd_witsml_flattened))
# _w_sensortypes_select = pn.widgets.Select(name='Select Sensor Type')

# # create the DataFrame and plot
# _pd_real_time_plot = pd.DataFrame({
#     'Value': [1, 2, 3, 4, 5],
#     'Date': pd.date_range('2022-01-01', periods=5, freq='D')
# })

# plot = _pd_real_time_plot.hvplot(height=1000,
#     width=500, title='Real Time',
#     x='Value',
#     y=['Date'], 
#     ylim=(min(_pd_real_time_plot['Date']), max(_pd_real_time_plot['Date']))
# ).opts(invert_yaxis=True)

# # define the update function
# def update_plot(event):
#     sensorid = _get_sensorid(_pd_witsml_flattened)
#     id, data = _make_pd_real_time_plot(sensorid)
#     new_plot = create_plot(data, _pd_witsml_flattened)
#     plot.object = new_plot.object

# # bind the update function to the widget values
# _w_wellbores_select.param.watch(update_plot, 'value')
# _w_sensortypes_select.param.watch(update_plot, 'value')

# # display the widgets and plot
# pn.Row(_w_wellbores_select, _w_sensortypes_select, plot).servable()

In [ ]:
# import pandas as pd
# import numpy as np
# import hvplot.pandas
# import panel as pn

# # create dataset d1
# d1 = pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10),
#     'y2': np.random.rand(10)
# })

# # create dataset d2 with different column names
# d2 = pd.DataFrame({
#     'x': np.arange(10),
#     'z1': np.random.rand(10),
#     'z2': np.random.rand(10)
# })

# # plot d1 using hvplot
# plot = d1.hvplot.line(x='x', y=['y1', 'y2'])

# # define the update function
# def update_plot(event):
#     #global plot
#     newplot = d2.hvplot.line(x='x', y=['z1', 'z2'], title="d2")
#     plot = newplot
#     plot

# # create the button widget
# button = pn.widgets.Button(name='Update Plot')


# indicator = pn.indicators.LoadingSpinner(value=False, size=25)
# text = pn.widgets.TextInput(value='Start')
 
# def update_indicator(event, clicks):
#     if not event:
#         return
   
#     indicator.value = not indicator.value

#     test = pn.Column(button, (d2.hvplot.line(x='x', y=['z1', 'z2'], title="d2")), indicator, text).servable()
#     test.servable
#     text.value = f"you have clicked me {clicks} times"
#     plot.title = "hjhkj"
 
# # pn.bind(update_indicator, button, button.param.clicks, watch=True)

# # pn.bind(update_plot, button, watch=True)

# # bind the update function to the button click event
# # button.on_click(update_plot)

# # display the button and plot using Panel
# # pn.Column(button, plot, indicator, text).servable()
# hvplot.bind(update_plot, button).interactive().hvplot.line(title="d2")


In [ ]:
# import hvplot.pandas  # noqa
# import pandas as pd
# import panel as pn

# pn.extension()


# def function_to_generate_dataframe(x):
#     # x only used to trigger the function
#     df =  pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10),
#     'y2': np.random.rand(10)
# })
#     # df["C"] = df["C"].round()
#     return df


# widget = pn.widgets.IntSlider(end=100)
# hvplot.bind(function_to_generate_dataframe, widget).interactive().hvplot.line()


In [ ]:
# import hvplot.pandas
# import pandas as pd
# import panel as pn
# import numpy as np

# pn.extension()

# # create dataset d1
# d1 = pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10)
# })


# # create dataset d2 with different column names
# d2 = pd.DataFrame({
#     'x': np.arange(10),
#     'y1': np.random.rand(10)
#     })

# def function_to_generate_dataframe(x):
#     # x only used to trigger the function
#     global d1, d2
#     if x == 'd1':
#         df = d1
#     else:
#         df = d2
    
#     return df.hvplot.line(x='x', title=x)

# widget = pn.widgets.Select(options=['d1', 'd2'])
# hvplot.bind(function_to_generate_dataframe, widget).interactive().hvplot.line()

In [ ]:
#WORKS


# # create dataset d1
# d1 = _pd_real_time_plot


# # create dataset d2 with different column names
# d2 = _pd_real_time_plot

# def function_to_generate_dataframe2(x):
#     # x only used to trigger the function
#     global d1, d2
#     if x == 'd1':
#         df = d1
#     else:
#         df = d2
    
#     return df.hvplot.line(x='Value', title=x)

# widget2 = pn.widgets.Select(options=['d1', 'd2'])
# p = hvplot.bind(function_to_generate_dataframe2, widget2).interactive()
# p.hvplot().line()


In [ ]:
# ### WORKs as of 20.06 
# #tesing with new code and proper WORRKSSS YESSS
# # import hvplot.pandas
# from bokeh.models import DatetimeTickFormatter

# def master(wellbore, sensortype):
#     # print(wellbore, sensortype)
#     # print(type(sensorid))
#     # print(sensorid)
#     # s= int(sensorid)
#     # global num_rows

#     data, num_rows = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
#     plot_opts = dict(height=500, width=500, xlabel =dict(_w_sensortype.param.values().items())['value'], yformatter=DatetimeTickFormatter(days="%m/%d",
#                                             hours="%H",
#                                             minutes="%H:%M") )#'%Y-%m-%d %H:%M:%S')
    
#     # param.depends(_w_sensortype)(scatter_plot.plot)
#     print("finished in master")
    


#     return data#, plot


# def _combine_datasets(wellbore,sensortype):
#     sensorid = _get_sensorid(_pd_witsml_flattened,wellbore,sensortype)

#     id, data = _make_pd_real_time_plot(int(sensorid))

#     # test= pd.DataFrame({'DescriptionDateStart':pd.to_datetime(['2022-11-24 05:27:00', '2022-11-25 05:27:10', '2022-11-27 05:27:20', '2022-11-29 05:30:10']),
#     # 'DescriptionDateEnd':pd.to_datetime(['2022-11-25 05:27:00', '2022-11-26 05:27:10', '2022-11-28 05:27:20', '2022-12-01 05:30:10']),

#     # 'x_value': pd.Series([50000000000000000.00, 50000000000000000.00, 50000000000000000.00,50000000000000000.00]),
#     # 'Description': pd.Series(["yo","hello","everyone","here"]) })


#     # combined_df = pd.concat([data, test], axis=0, ignore_index=True)
#     # data = combined_df
#     num_rows = len(data)
#     min_d = min(data['Date'])
#     max_d = max(data['Date'])

#     # data['test_date'] = data['Date'].apply(lambda x: x.timestamp())
#     # data['proper_date'] = pd.to_datetime(data['Date'])


#     return data, num_rows

# # def _get_num_rows(wellbore,sensortype):
# #     data, num_rows = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
# #     return num_rows







# # # get the current value of the try_sensor widget
# # def callback_sensortype(event):
# #     sensortype = event
# #     return sensortype

# # # bind the callback function to the _w_sensortypes_select widget
# # _w_sensortype.param.watch(callback_sensortype, 'value')
# # # try_sensor

# # def get_chosen_sensortype():
# #     # get the current value of the try_sensor widget using the callback_sensortype function
# #     sensortype = callback_sensortype( _w_sensortype.value)
# #     print("sensortype")
# #     print(sensortype)
# #     return sensortype


# def update_title():
#     create_plot().title="new"
    

# def callback_sensortype(event):
#     # get the current value of the _w_sensortype widget
#     print(type(event))
#     sensortype = event.new
#     print(type(sensortype))
#     # plot.title="new"
#     # _w_sensortype.observe(update_title, 'value')
#     # _w_sensortype.select(update_title, 'value')
#     # print(type(_w_sensortype.__getattribute__('value')))
#     # print(_w_sensortype.__getattribute__('value'))
#     return sensortype

# # bind the callback function to the _w_sensortype widget
# _w_sensortype.param.watch(callback_sensortype, 'value')




# def _make_interactive():
#     _interactive = hvplot.bind\
#     (master,_w_wellbore, _w_sensortype)\
#     .interactive()
#     # Call the function to update the labels
#     print("finished")

#     return _interactive, dict(_w_sensortype.param.values().items())['value']
# # def master():
# #     function_to_generate_dataframe(_w_wellbore, _w_sensortype) 


# # def _make_interactive():
# #     _interactive = hvplot.bind\
# #     (master)\
# #     .interactive()
# #     print("finished")
# #     return _interactive#, _w_sensortype.param.get_param_values().__getitem__(24)[1]

# _interactive, s = _make_interactive()

# # get_chosen_sensortype()
# # _w_sensortype

# # callback_sensortype(_w_sensortype)

# # _make_interactive().head(5)

# # plot = create_plot(_make_interactive()[0],_make_interactive()[1])
# # plot

In [ ]:
### WORKs as of 20.06 - COPY for test
#tesing with new code and proper WORRKSSS YESSS
# import hvplot.pandas
from bokeh.models import DatetimeTickFormatter

def master(wellbore, sensortype):
    # print(wellbore, sensortype)
    # print(type(sensorid))
    # print(sensorid)
    # s= int(sensorid)
    # global num_rows

    data = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
    plot_opts = dict(height=500, width=500, xlabel =dict(_w_sensortype.param.values().items())['value'], yformatter=DatetimeTickFormatter(days="%m/%d",
                                            hours="%H",
                                            minutes="%H:%M") )#'%Y-%m-%d %H:%M:%S')
    
    # param.depends(_w_sensortype)(scatter_plot.plot)
    print(str(_w_sensortype))
    print(dict(_w_sensortype.param.values().items())['value'])
    print("finished in master")
    # create_plot(data)
    # plot = create_plot(data)

    return data#, plot


def _combine_datasets(wellbore,sensortype):
    sensorid = _get_sensorid(_pd_witsml_flattened,wellbore,sensortype)

    id, data = _make_pd_real_time_plot(int(sensorid))

    # test= pd.DataFrame({'DescriptionDateStart':pd.to_datetime(['2022-11-24 05:27:00', '2022-11-25 05:27:10', '2022-11-27 05:27:20', '2022-11-29 05:30:10']),
    # 'DescriptionDateEnd':pd.to_datetime(['2022-11-25 05:27:00', '2022-11-26 05:27:10', '2022-11-28 05:27:20', '2022-12-01 05:30:10']),

    # 'x_value': pd.Series([50000000000000000.00, 50000000000000000.00, 50000000000000000.00,50000000000000000.00]),
    # 'Description': pd.Series(["yo","hello","everyone","here"]) })


    # combined_df = pd.concat([data, test], axis=0, ignore_index=True)
    # data = combined_df
    # num_rows = len(data)
    # min_d = min(data['Date'])
    # max_d = max(data['Date'])

    # data['proper_date'] = pd.to_datetime(data['Date'])


    return data#, num_rows

# def _get_num_rows(wellbore,sensortype):
#     data, num_rows = _combine_datasets(wellbore=wellbore, sensortype=sensortype)
#     return num_rows







# # get the current value of the try_sensor widget
# def callback_sensortype(event):
#     sensortype = event
#     return sensortype

# # bind the callback function to the _w_sensortypes_select widget
# _w_sensortype.param.watch(callback_sensortype, 'value')
# # try_sensor

# def get_chosen_sensortype():
#     # get the current value of the try_sensor widget using the callback_sensortype function
#     sensortype = callback_sensortype( _w_sensortype.value)
#     print("sensortype")
#     print(sensortype)
#     return sensortype


def update_title():
    create_plot().title="new"
    

def callback_sensortype(event):
    # get the current value of the _w_sensortype widget
    print(type(event))
    sensortype = event.new
    print(type(sensortype))
    # plot.title="new"
    # _w_sensortype.observe(update_title, 'value')
    # _w_sensortype.select(update_title, 'value')
    # print(type(_w_sensortype.__getattribute__('value')))
    # print(_w_sensortype.__getattribute__('value'))
    return sensortype

# bind the callback function to the _w_sensortype widget
_w_sensortype.param.watch(callback_sensortype, 'value')




def _make_interactive():
    _interactive = hvplot.bind\
    (master,_w_wellbore, _w_sensortype)\
    .interactive()
    # Call the function to update the labels
    print("finished")



    return _interactive#, dict(_w_sensortype.param.values().items())['value']
# def master():
#     function_to_generate_dataframe(_w_wellbore, _w_sensortype) 


# def _make_interactive():
#     _interactive = hvplot.bind\
#     (master)\
#     .interactive()
#     print("finished")
#     return _interactive#, _w_sensortype.param.get_param_values().__getitem__(24)[1]

_interactive = _make_interactive()

# get_chosen_sensortype()
# _w_sensortype

# callback_sensortype(_w_sensortype)

# _make_interactive().head(5)

# plot = create_plot(_make_interactive()[0],_make_interactive()[1])
# plot

In [ ]:
# testing 24.06
df_test = _pd_real_time_plot
df_test['Date_Seconds'] = df_test['Date'].apply(lambda x: x.timestamp())
df_test['index'] = df_test['Date_Seconds'] 


_df_rectangles = pd.DataFrame({
        'x': [2000000000000.00, 2000000000000.00, 2000000000000.00],
        'ystart': [pd.Timestamp('2022-11-24 05:27:00'), pd.Timestamp('2022-11-29 05:25:00'), pd.Timestamp('2022-12-25 05:29:00')],
        'yend': [pd.Timestamp('2022-11-25 05:27:00'), pd.Timestamp('2022-12-02 05:27:00'), pd.Timestamp('2022-12-29 05:27:00')],
        'width': [20000000000000.00, 20000000000000.00, 20000000000000.00],
        'label': ['Rectangle 1 - U', 'Rectangle 2 - D', 'Rectangle 3 - U']
    })

_df_rectangles['ystart_seconds'] =  _df_rectangles['ystart'].apply(lambda x: x.timestamp())
_df_rectangles['yend_seconds'] =  _df_rectangles['yend'].apply(lambda x: x.timestamp())
_df_rectangles['index_extra'] =  _df_rectangles['ystart_seconds']

_df_combined = df_test.set_index('index').join(_df_rectangles.set_index('index_extra'))
_df_combined.head(5)

In [ ]:
line_plot = _df_combined.hvplot.line(x='Date_Seconds', y=['Value'])
scatter_plot = _df_combined.hvplot(x='Date_Seconds', y=['Value'], kind='scatter')
line_plot + scatter_plot

In [80]:


# Create a line plot of Date_Seconds and Value
line_plot = _df_combined.hvplot.line(x='Date_Seconds', y=['Value'])

# Create a Rect element for each rectangle in _df_combined
rects = []
for i, row in _df_combined.iterrows():
    if not pd.isna(row['ystart_seconds']):
        rect = hv.Rectangles((row['ystart_seconds'], 1000000000, row['yend_seconds']), row['Value'], row['label'])
        rects.append(rect)

# Create a plot with the line plot and Rect elements overlaid
# plot = hv.Overlay([line_plot] + rects)
rect_plots = hv.Overlay(rects)
# rect_plots
# Set the x-axis label to 'Date' and the y-axis label to 'Value'
line_plot*rect_plots
# Display the plot
# plot

In [ ]:


# Create a Rect element for each rectangle in _df_combined
rects = []
for i, row in _df_combined.iterrows():
    if not pd.isna(row['ystart_seconds']):
        rect = hv.Rect((row['ystart_seconds'], 0, row['yend_seconds']), row['Value'], row['label'])
        rects.append(rect)

# Create a line plot of Date_Seconds and Value
line_plot = _df_combined.hvplot.line(x='Date_Seconds', y='Value')

# Create a plot with the line plot and Rect elements overlaid
rect_plots = hv.Overlay(rects)
plot = line_plot * rect_plots

# Set the x-axis label to 'Date' and the y-axis label to 'Value'
plot = plot.opts(xlabel='Date', ylabel='Value')

# Display the plot
plot

In [ ]:
_sensortype = _w_sensortype
_wellbore = _w_wellbore
# print(_sensortype)

line_plot = _df_combined.hvplot(height=1000,

    width=500, title= 'Real Time data for ' + " ",# + str(dict(_w_sensortype.param.values().items())['value']),# + _wellbore,
    x='Value',
    xlabel = _sensortype,
    # hover_cols=['Description'],
    # y=['Date'],
    y=['Date_Seconds'],

    autorange="y"
    # value_label = 'Date'
    # ,ylim=(min(df['Date']), max(df['Date']))#,
)
# plots = []

# # Add the rectangles to the plot
# for rectangle in _df_combined:
#     # print(rectangle)
#     # Define the vertices of the rectangle
#     vertices = [(_df_combined['x'], _df_combined['ystart_seconds']), (_df_combined['x'] + _df_combined['width'], _df_combined['ystart_seconds']),
#                 (_df_combined['x'] + _df_combined['width'], _df_combined['ystart_seconds'] + (_df_combined['ystart_seconds'] - _df_combined['yend_seconds'])),
#                 (_df_combined['x'], _df_combined['ystart_seconds'] + (_df_combined['ystart_seconds'] - _df_combined['yend_seconds']))]
    

# # Convert the y-coordinates to float values
# vertices = [(x,ystart) for x, ystart in vertices]

# # set colors of rectanlges
# if '- D' in _df_combined['label']:
#     fill_color = 'red'
# elif '- U' in _df_combined['label']:
#     fill_color = 'green'
# else:
#     fill_color = 'orange'

# # Create a polygon for the rectangle
# polygon = hv.Polygons([vertices]).opts(alpha=0.5, color = fill_color)
# # print(vertices)
# # Add the polygon to the list of plots
# plots.append(polygon)

# Add the label to the rectangle
# label = hv.Text(_df_combined['x'] + _df_combined['width'] / 2, _df_combined['ystart'] + (_df_combined['ystart'] - _df_combined['yend']) / 2, _df_combined['label']).opts(color='black', xaxis=None, height =100, width = WIDTH) # yaxis=None,

# Add the label to the list of plots
# plots.append(label)
scatter = _df_combined.hvplot(kind='scatter', y=['ystart_seconds'])
# Combine all the plots into a single plot

combined_plot = line_plot + hv.Overlay(scatter) #hvploy(y='ystart_seconds')#hv.Overlay(plots)
# combined_plot.opts(shared_axes=True)
combined_plot

In [ ]:
print(str(_w_sensortype))
print(dict(_w_sensortype.param.values().items())['value'])

In [ ]:
create_plot(_interactive)

In [ ]:


# Create some sample data
df1 = pd.DataFrame({'x': np.arange(10), 'y': np.random.rand(10)})
df2 = pd.DataFrame({'x': np.arange(10), 'y': np.random.rand(10)})

# Create two plots using hvplot
plot1 = df1.hvplot.line(x='x', y='y')
plot2 = df2.hvplot.line(x='x', y='y')

# Combine the plots into a layout with shared y-axis
layout = hv.Layout([plot1, plot2]).cols(2).opts(shared_axes=True)

# Display the layout
layout

In [ ]:

def _make_interactive_create_plot():
    data =  master(_w_wellbore, _w_sensortype)
    _int_plot = hvplot.bind\
    (create_plot, data)\
    .interactive()
    # Call the function to update the labels
    print("finished")
_make_interactive_create_plot()

In [ ]:
hv.help(hv.Polygons)

In [ ]:

plot = create_plot(_interactive)
plot
# # Create two dataframes
# df1 = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
# df2 = pd.DataFrame({'x': [1, 2, 3], 'y': [7, 8, 9], 'desc': ["yo", "hello", "there"]})

# # Create two plots
# plot1 = df1.hvplot.line(x='x', y='y')
# plot2 = df2.hvplot.scatter(x='x', y='y', hover_cols = ['desc'],color= "#f16a6f")

# # Overlay the plots
# overlayed_plot = plot1 * plot2

# # Display the overlayed plot
# overlayed_plot

In [ ]:
update_labels(plot, "well", s)


In [ ]:
dict(_w_sensortype.param.values().items())['value']

In [ ]:
### THIS ONE - edit with pd.Timestamp() instead
# Define the coordinates of the rectangles
rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]


# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['ystart'] = rectangle['ystart'].timestamp()
    rectangle['yend'] = rectangle['yend'].timestamp()


# setting global values for styling graps
HEIGHT = 500
WIDTH = 500
# Create a line plot
line_plot = _interactive.hvplot(height=HEIGHT,
    width=500, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
    x='Value',
    # xlabel = _sensortype,
    # hover_cols=['Description'],
    y=['Date']# ,
    # value_label = 'Date'
    # ,ylim=(min(df['Date']), max(df['Date']))#,
)

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['ystart']), (rectangle['x'] + rectangle['width'], rectangle['ystart']),
                (rectangle['x'] + rectangle['width'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend'])),
                (rectangle['x'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']))]
    

    # Convert the y-coordinates to float values
    vertices = [(x,ystart) for x, ystart in vertices]

    # set colors of rectanlges
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5, color = fill_color)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']) / 2, rectangle['label']).opts(color='black', xaxis=None, height =HEIGHT, width = WIDTH) # yaxis=None,
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot + hv.Overlay(plots)

# Display the plot
combined_plot

#TODO make plot object and updatexlable attribute?



In [ ]:
import pandas as pd
import holoviews as hv

# Define the coordinates of the rectangles
rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]

# Set the plot options
plot_opts = dict(height=500, width=500, xaxis=None, yformatter='%Y-%m-%d %H:%M:%S')

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['ystart'].timestamp()), 
                (rectangle['x'] + rectangle['width'], rectangle['ystart'].timestamp()),
                (rectangle['x'] + rectangle['width'], rectangle['yend'].timestamp()),
                (rectangle['x'], rectangle['yend'].timestamp())]

    # Set the fill color of the rectangle based on the label
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).options(alpha=0.5, color=fill_color)

    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['yend'] - rectangle['ystart']) / 2, rectangle['label']).options(color='black', xaxis=None)

    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = hv.Overlay(plots).options(**plot_opts)

# Display the plot
combined_plot
## LOOK HEREE

In [ ]:
combined_plot

In [ ]:
### THIS ONE COPY - works for timestamp() 
#  # Define the coordinates of the rectangles

# from bokeh.plotting import figure, output_file, save, Show
# from bokeh.models import DatetimeTickFormatter
# import datetime as dt
# import datetime as dt

rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-10 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]


# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['ystart'] = rectangle['ystart'].timestamp()
    rectangle['yend'] = rectangle['yend'].timestamp()


# setting global values for styling graps
# HEIGHT = 500
# WIDTH = 500
# Create a line plot
line_plot = _interactive.hvplot(height=HEIGHT,
    width=500, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
    x='Value',
    # xlabel = _sensortype,
    # hover_cols=['Description'],
    y=['Date']# ,
    # value_label = 'Date'
    # ,ylim=(min(df['Date']), max(df['Date']))#,
)

# Create a list to store the polygons and text labels
plots = []



# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['ystart']), (rectangle['x'] + rectangle['width'], rectangle['ystart']),
                (rectangle['x'] + rectangle['width'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend'])),
                (rectangle['x'], rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']))]
    

    # Convert the y-coordinates to float values
    vertices = [(x,ystart) for x, ystart in vertices]

    # set colors of rectanlges
    if '- D' in rectangle['label']:
        fill_color = 'red'
    elif '- U' in rectangle['label']:
        fill_color = 'green'
    else:
        fill_color = 'orange'
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5, color = fill_color)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)

    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['ystart'] + (rectangle['ystart'] - rectangle['yend']) / 2, rectangle['label']).opts(color='black', xaxis=None, height =HEIGHT, width = WIDTH) # yaxis=None,
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot.options(**plot_opts) + hv.Overlay(plots)#.options(**plot_opts)

# Display the plot
combined_plot




In [ ]:
_interactive = hvplot.bind\
    (function_to_generate_dataframe, _w_wellbores_select(_pd_witsml_flattened), _w_sensortype, lambda param1, param2: _w_another_function(param1, param2)) \
    .interactive()

In [ ]:
bound_functions

In [ ]:
import pandas as pd
import hvplot.pandas

# Define the coordinates of the rectangles
rectangles = [
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-24 05:27:00'), 'yend':pd.Timestamp('2022-11-25 05:27:00') ,'width': 20000000000000.00,  'label': 'Rectangle 1 - U'}, #'height': 200000.00,
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-11-29 05:25:00'), 'yend': pd.Timestamp('2022-12-02 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 2 - D'},
    {'x': 2000000000000.00, 'ystart': pd.Timestamp('2022-12-25 05:29:00'), 'yend': pd.Timestamp('2022-12-29 05:27:00'),'width': 20000000000000.00,  'label': 'Rectangle 3 - U'}
]

# Convert the rectangles to a Pandas DataFrame
df = pd.DataFrame(rectangles)

# Create the fill_color column based on the label of each rectangle
# df['fill_color'] = ['red' if '- D' in label else 'green' if '- U' in label else 'orange' for label in df['label']]

# Define the plot options
plot_opts = dict(height=500, width=500, xaxis=None, yformatter='%Y-%m-%d %H:%M:%S')

# Create the area plot
area_plot = df.hvplot.area(x='x', alpha=0.5, line_color=None, hover_alpha=1.0, hover_line_color='black').options(**plot_opts)

# Display the plot
area_plot

In [ ]:
area_plot

In [ ]:
combined_plot

In [ ]:


# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': '2022-11-24 05:27:00', 'width': 20000000, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': '2022-11-24 05:25:00', 'width': 20000000, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': '2022-11-24 05:29:00', 'width': 20000000, 'height': 3, 'label': 'Rectangle 3'}

]
# df['Date'] = df['Date'].apply(lambda x: x.timestamp())

# Create a line plot
line_plot = _interactive.hvplot(height=500,
    width=250, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
    x='Value',
    # xlabel = _sensortype,
    # hover_cols=['Description'],
    y='Date'# ,
    # value_label = 'Date'
    # ,ylim=(min(df['Date']), max(df['Date']))#,
)

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np

# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': pd.Timestamp('2022-11-24 05:27:00'), 'width': 20000000, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': pd.Timestamp('2022-11-24 05:25:00'), 'width': 20000000, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': pd.Timestamp('2022-11-24 05:29:00'), 'width': 20000000, 'height': 3, 'label': 'Rectangle 3'}
]

# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['y'] = rectangle['y'].timestamp()

# Create a line plot
df = pd.DataFrame({'Value': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'Date': pd.date_range('2022-11-24 05:25:00', periods=10, freq='T')})
line_plot = df.hvplot.line(x='Value', y='Date')

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Convert the y-coordinates to float values
    vertices = [(x, float(y)) for x, y in vertices]
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    print(vertices)
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    # label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    # plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np

# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': pd.Timestamp('2022-11-24 05:27:00'), 'width': 20000000, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': pd.Timestamp('2022-11-24 05:25:00'), 'width': 20000000, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': pd.Timestamp('2022-11-24 05:29:00'), 'width': 20000000, 'height': 3, 'label': 'Rectangle 3'}
]

# Convert the y values to Unix timestamps
for rectangle in rectangles:
    rectangle['y'] = rectangle['y'].timestamp()

# Create a line plot with Unix timestamp y values
df = pd.DataFrame({'Value': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'Date': pd.date_range('2022-11-24 05:25:00', periods=10, freq='T')})
df['Date'] = df['Date'].apply(lambda x: x.timestamp())
line_plot = df.hvplot.line(x='Value', y='Date')


# line_plot = _interactive.hvplot(height=500,
#     width=250, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
#     x='Value',
#     # xlabel = _sensortype,
#     # hover_cols=['Description'],
#     y=['Date']# ,
#     # value_label = 'Date'
#     # ,ylim=(min(df['Date']), max(df['Date']))#,
# )



# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Convert the y-coordinates to float values
    vertices = [(x, float(y)) for x, y in vertices]
    print(vertices)
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

In [ ]:
df.head(10)

In [ ]:
# Define the coordinates of the rectangle
x1, y1 = 1, 4
x2, y2 = 3, 6

# Create a dataframe with the coordinates of the rectangle
df = pd.DataFrame({'x': [x1, x2, x2, x1], 'y': [y1, y1, y2, y2]})

# Create an area plot for the rectangle
plot = df.hvplot.area(x='x', y='y', alpha=0.5)

# Display the plot
plot

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import matplotlib.dates as mdates

# Define the coordinates of the rectangles
y1, y2 = 4, 6
df = pd.DataFrame({
    'starttime': ['2022-11-24 05:27:00', '2022-11-24 05:27:20'],
    'endtime': ['2022-11-24 05:30:10', '2022-11-24 05:32:00'],
    'label': ['Rectangle 1', 'Rectangle 2']
    
})

# Convert the starttime and endtime columns to datetime objects
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])

# Create a new column with the duration of each rectangle
# df['duration'] = df['endtime'] - df['starttime']

# Create a new column with the y2 coordinate of each rectangle
# df['y2'] = df['starttime']

# Create a new column with the y1 coordinate of each rectangle
# df['y1'] = df['y2'] - df['duration']

# Create an area plot for the rectangles with inverted axes
plot = df.hvplot.area(x='starttime', y='starttime', x2='endtime', y2='endtime', alpha=0.5, invert_axes=True)

# Convert the datetime objects to numeric values
x_values = mdates.date2num(df['starttime'].to_numpy())
y_values = mdates.date2num(df['starttime'].to_numpy())

# Add text labels to the plot
labels = hv.Text('2022-11-24 05:27:00', '2022-11-24 05:30:10', "test")#.opts(invert_axes=True, color='black')
hv.help(hv.Text)

In [ ]:
%pip install mpld3

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv

# Define the coordinates of the rectangles
rectangles = [
    {'x': 1, 'y': 4, 'width': 2, 'height': 2, 'label': 'Rectangle 1'},
    {'x': 4, 'y': 5, 'width': 3, 'height': 1, 'label': 'Rectangle 2'},
    {'x': 7, 'y': 3, 'width': 1, 'height': 3, 'label': 'Rectangle 3'}
]

# Create a line plot
df = pd.DataFrame({'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'y': [3, 4, 5, 6, 5, 4, 3, 2, 1, 2]})
line_plot = df.hvplot.line(x='x', y='y')

# Create a list to store the polygons and text labels
plots = []

# Add the rectangles to the plot
for rectangle in rectangles:
    # Define the vertices of the rectangle
    vertices = [(rectangle['x'], rectangle['y']), (rectangle['x'] + rectangle['width'], rectangle['y']),
                (rectangle['x'] + rectangle['width'], rectangle['y'] + rectangle['height']),
                (rectangle['x'], rectangle['y'] + rectangle['height'])]
    
    # Create a polygon for the rectangle
    polygon = hv.Polygons([vertices]).opts(alpha=0.5)
    
    # Add the polygon to the list of plots
    plots.append(polygon)
    
    # Add the label to the rectangle
    label = hv.Text(rectangle['x'] + rectangle['width'] / 2, rectangle['y'] + rectangle['height'] / 2, rectangle['label']).opts(color='black')
    
    # Add the label to the list of plots
    plots.append(label)

# Combine all the plots into a single plot
combined_plot = line_plot * hv.Overlay(plots)

# Display the plot
combined_plot

In [ ]:
hv.help(hv.Text)

In [ ]:
import pandas as pd
import hvplot.pandas

# Define the coordinates of the rectangles
y1, y2 = 4, 6
df = pd.DataFrame({
    'starttime': ['2022-11-24 05:27:00', '2022-11-24 05:27:20'],
    'endtime': ['2022-11-24 05:30:10', '2022-11-24 05:32:00'],
    'y1': [y1, y1],
    'y2': [60, 60]
})

# Convert the starttime and endtime columns to datetime objects
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])

# Create a new column with the duration of each rectangle
df['duration'] = df['endtime'] - df['starttime']

# Create a new column with the x2 coordinate of each rectangle
df['x2'] = df['starttime']

# Create a new column with the x1 coordinate of each rectangle
df['x1'] = df['x2'] - df['duration']

# Create an area plot for the rectangles
plot = df.hvplot.area(x=['y1', 'y2'], y='starttime', y2='endtime', alpha=0.5)

# Display the plot
plot

In [ ]:
import pandas as pd
import hvplot.pandas

# Define the coordinates of the rectangles
y1, y2 = 4, 6
df = pd.DataFrame({
    'starttime': ['2022-11-24 05:27:00', '2022-11-24 05:27:20'],
    'endtime': ['2022-11-24 05:30:10', '2022-11-24 05:32:00'],
    'y1': [y1, y1],
    'y2': [60, 60]
})

# Convert the starttime and endtime columns to datetime objects
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])

# Create a new column with the duration of each rectangle
df['duration'] = df['endtime'] - df['starttime']

# Create a new column with the y2 coordinate of each rectangle
df['y2'] = df['starttime']

# Create a new column with the y1 coordinate of each rectangle
df['y1'] = df['y2'] - df['duration']

# Create an area plot for the rectangles with inverted axes
plot = df.hvplot.area(x='y1', y='starttime', x2='y2', y2='endtime', alpha=0.5)#.opts(inverted=True)

# Display the plot
plot

In [ ]:
_interactive.head(5)

In [ ]:
combined_plot

In [ ]:
pn.Row(plot, plot.Scatter)

In [ ]:
print(plot.count())

In [ ]:
print(len(i.result()))


In [ ]:
plot.Scatter

In [ ]:
_interactive, s =_make_interactive()
# _interactive.head(20)

plot = _interactive.hvplot(height=1000,
        width=500, title= 'Real Time data for ',# + _wellbore + " " + _sensortype,
        x='Value',
        # xlabel = _sensortype,
        y=['Date'] 
        # ,ylim=(min(df['Date']), max(df['Date']))#,
    )
scatter_plot = _interactive.hvplot(y=['DescriptionDate'], hover_cols=['Description'], kind='scatter')
combined_plot  =plot * scatter_plot
# combined_plot

In [ ]:
# combined_plot
_interactive.hvplot(y=['DescriptionDate'], hover_cols=['Description'], kind='scatter')

In [ ]:
# _interactive[!_interactive.isna()]
             
_interactive[_interactive['Date'].notna()].head(5)


In [ ]:
import random
import string


# Create a range of timestamps
timestamps = pd.date_range(start='2022-01-01', end='2022-01-05', freq='H')



# Plot the DataFrame with data points for the 'description' column
plot_test = _interactive.hvplot(height=1000,
        width=500, title= 'Real Time data for ' ,
        x='Value',
        y=['Date'] 
        # ,ylim=(min(df['Date']), max(df['Date']))#,
    )
# Display the plot
plot_test

In [ ]:
plot

In [ ]:
# new_interactive_obj = _make_interactive()[0].observe(create_plot, _make_interactive()[0],_make_interactive()[1])
_interactive.head(5)


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the function that will be used in the interactive object
def my_function(x, y):
    return x + y

# Create the widgets for the interactive object
x_widget = widgets.IntSlider(min=0, max=10, step=1, value=5, description='x:')
y_widget = widgets.IntSlider(min=0, max=10, step=1, value=5, description='y:')

# Create the interactive object using the function and widgets
interactive_obj = widgets.interactive(my_function, x=x_widget, y=y_widget)

# Define a function to update the title of the interactive object
def update_title(change):
    interactive_obj.title = f"x={x_widget.value}, y={y_widget.value}, result={interactive_obj.result}"

# Watch for changes in the value of the x_widget and y_widget, and update the title of the interactive object
x_widget.observe(update_title, 'value')
y_widget.observe(update_title,   'value')

# Display the interactive object
display(interactive_obj)

In [ ]:
### TEST MAKE CLASS


class Interactive(param.parameterized):
    # import hvplot.pandas
    def function_to_generate_dataframe(wellbore, sensortype):
        print(wellbore, sensortype)
        sensorid = _get_sensorid(_pd_witsml_flattened,wellbore,sensortype)
        print(type(sensorid))
        print(sensorid)
        s= int(sensorid)
        id, data = _make_pd_real_time_plot(s)
        min_d = min(data['Date'])
        max_d = max(data['Date'])
        return data

    _w_sensortype = _w_sensortypes_select(_pd_witsml_flattened)


    # # get the current value of the try_sensor widget
    # def callback_sensortype(event):
    #     sensortype = event
    #     return sensortype

    # # bind the callback function to the _w_sensortypes_select widget
    # _w_sensortype.param.watch(callback_sensortype, 'value')
    # # try_sensor

    # def get_chosen_sensortype():
    #     # get the current value of the try_sensor widget using the callback_sensortype function
    #     sensortype = callback_sensortype( _w_sensortype.value)
    #     print("sensortype")
    #     print(sensortype)
    #     return sensortype
    def callback_sensortype(event):
        # get the current value of the _w_sensortype widget
        print(type(event))
        sensortype = event.new
        print(type(sensortype))
        # print(type(_w_sensortype.__getattribute__('value')))
        # print(_w_sensortype.__getattribute__('value'))
        return sensortype

    # bind the callback function to the _w_sensortype widget
    _w_sensortype.param.watch(callback_sensortype, 'value')



    def _make_interactive():
        _interactive = hvplot.bind\
        (function_to_generate_dataframe, _w_wellbores_select(_pd_witsml_flattened), _w_sensortype) \
        .interactive()
        return _interactive, _w_sensortype.param.get_param_values().__getitem__(24)[1]


    # get_chosen_sensortype()
    # _w_sensortype

    # callback_sensortype(_w_sensortype)

    # _make_interactive().head(5)


In [ ]:
callback_sensortype(_w_sensortype)

In [ ]:
_w_sensortype

In [ ]:
widget = _w_sensortype.param

value = widget.get_param_values().__getitem__(24)[1]
value
# values.__dict__()
# value = widget.value()

In [ ]:
_make_interactive().head(5)

In [ ]:
try_sensor = _w_sensortypes_select(_pd_witsml_flattened)


In [ ]:
try_sensor

In [ ]:
create_plot(_make_interactive()[0],_make_interactive()[1])

In [ ]:
# TItle of plot is not updating bc you don't trigger _create_plot(), only change the ds within it

In [ ]:
import pandas as pd
import hvplot.pandas
import param

# Define the interactive object with a select widget
class MyObject(param.Parameterized):
    select_widget = param.Selector(objects=['Option 1', 'Option 2'], default='Option 1', label='Select an option')

    def plot(self):
        # Generate a sample DataFrame
        df = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})

        # Create the plot with hvplot
        plot = df.hvplot.scatter(x='x', y='y', title=self.select_widget)

        # Watch for changes in the value of the select widget and update the title of the plot
        def update_title(event):
            plot.title = event.new
        self.param.watch(update_title, 'select_widget')

        return plot

# Create an instance of the interactive object and display the plot
my_object = MyObject()
my_object.plot()

In [ ]:
_w_wellbores_select(_pd_witsml_flattened)

In [ ]:
_w_wellbores_select(_pd_witsml_flattened).param.values()

In [ ]:


df = pn.rx(data)
plot = df.hvplot(height=1000,
    width=500, title='Real Time',
    x='Value',
    y=['Date'], 
    ylim=(min(df['Date']), max(df['Date']))    # ).opts(invert_yaxis=True)
)



In [ ]:
df = data
e = pn.widgets.IntSlider(value=2, start=1, end=10, name="e").rx()
pn.Row(pn.rx(df.hvplot.line)(line_width=e))

In [ ]:
min(data['Date'])

In [ ]:
create_plot(data, _pd_witsml_flattened)

In [ ]:
print(wellbore, sensor)

In [ ]:
_get_sensorid(_pd_witsml_flattened)

In [ ]:
_pd_witsml_flattened[(_pd_witsml_flattened['nameWellbore'] == 'NO 16/2-D-18') & (_pd_witsml_flattened['uid'] == 'BITDEP')]


In [ ]:
_pd_witsml_flattened[_pd_witsml_flattened['id'] == 881650803765]

In [ ]:
_pd_witsml_flattened

In [ ]:
_sel_sensorid

In [ ]:
id, data = _make_pd_real_time_plot(7119861774595595)
data

In [ ]:
test= pn.rx(_pd_real_time_plot)

head_widget = pn.widgets.IntSlider(name="Head", start=1, end=10)

test.head(head_widget)

In [ ]:

pn.ReactiveExpr(
    test.head(head_widget),  # only show a few rows to save some space
    widget_layout=pn.Column,
    widget_location="top",
)

In [ ]:
table = pn.widgets.Tabulator(test, page_size=10, pagination="remote")
pn.Column(_w_wellbores_select(_pd_witsml),  table)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st

from matplotlib.figure import Figure

data = np.random.normal(1, 1, size=100)
fig = Figure(figsize=(8,4))
ax = fig.subplots()
bins = pn.widgets.slider(value=20, min_value=10, max_value=30, step=1, label="Bins")
ax.hist(data, bins=bins)

st.pyplot(fig)